In [3]:
from scripts.gcd import gcd
from scripts.factor import prime_factors
from scripts.powmod import pow_mod

LCM of 84 and 108 is 756 == 84*108/gcd(84, 108) == 9072/12


# Utility Functions

In [4]:
def is_prime_simple(n):
    factors = prime_factors(n)
    if len(factors) == 1 and ((n, 1) in factors):
        return True
    return False

def mult_inverse(n, p):
    return pow(n, -1, p)

# Euler

In [5]:
def euler_square(a, p):
    '''
        a - an integer coprime to p
        p - a prime
    '''
    assert gcd(a, p) == 1
    assert is_prime_simple(p)
    exp = (p-1)//2
    num = a**exp % p
    if num == 1:
        print(f'{a}^({p}-1)//2 == 1 -> square')
        return True
    elif num == (p-1):
        print(f'{a}^({p}-1//2) != 1 -> non-square')
        return False

In [6]:
assert euler_square(2, 17)

2^(17-1)//2 == 1 -> square


# Legendre and Jacobi

In [44]:
# pseudo-code from https://primes.utm.edu/glossary/page.php?sort=JacobiSymbol
def jacobi(a, n):
    t = 1
    while a != 0:
        while a % 2 == 0:
            a /= 2
            r = n % 8
            if r == 3 or r == 5:
                t = -t
        a, n = n, a
        if a % 4 == n % 4 == 3:
            t = -t
        a %= n
    if n == 1:
        return t
    else:
        return 0

# Chinese Remainder Theorem

### generating table

In [7]:
import numpy as np

# p - num rows
p = 3
# q - num cols
q = 7
n = p*q
table = np.ones(shape=(p,q))
for i in range(n):
    table[i%p][i%q] = i 
print(table)

[[ 0. 15.  9.  3. 18. 12.  6.]
 [ 7.  1. 16. 10.  4. 19. 13.]
 [14.  8.  2. 17. 11.  5. 20.]]


## solving crt

In [8]:
def crt(a, b, p, q):
    '''
        eq1 -> x = (a % p)
        eq2 -> x = (b % q)
        
        solves the number x which goes in (a,b) in the crt table for p*q
    '''
    result = b % q
    # a + k*p for k = (0, ...) gives candidates to test that satisfy eq1
    # a + k*p = (b % q) is what we want to solve so subtract a
    result = (result - a) % q
    # k*p = (b-a % q), so find inverse of p mod q
    p_inverse = mult_inverse(p, q)
    # multiply both sides by p inverse
    result = (result*p_inverse) % q
    # result is the k value that solves the problem
    # answer is a + k*p mod p*q
    answer = (a + result*p) % (p*q)
    return answer

In [9]:
assert crt(6, 16, 13, 31) == 357

# Zero Knowledge Proofs

$n = p*q$

In [6]:
n = 1549411

$x=s^2\:mod\:n$

In [7]:
x = 1075922

$y=u^2\:mod\:n$

In [8]:
y= 1361157

## Step 2 - Bob responds with {0,1}

In [9]:
b = 0

## Step 3 - Alice responds accordingly
if b==0:
    $z=u$
    
else:
    $z=s*u$

In [10]:
z = 4467

## Step 4 - Bob verifies
if b==0:
    $z^2\equiv y\:mod\:n$
    
else:
    $z^2\equiv x*y\:mod\:n$

In [11]:
if b==0:
    print("b is 0, Bob responds: ",pow(z,2, n) == (y % n))
else:
    print("b is 1, Bob responds: ", pow(z,2, n) == (x*y % n))

b is 0, Bob responds:  True


# El Gamal

N = large prime (public)

a = Alice's private key

g = $p.e.\:mod\:N$

A = $g^a\:mod\:N$



### Set Up

In [194]:
N = 4460190157
g = 974586571
A = 1616165761

a = 95961

message = "CALL ME ISHMAEL" # CALL MEIS HMAE L

$m_1,m_2,...,m_n$ and we have cipher text:

$(r_i, t_i)\:=\:(g^{k_i},\:m_iA^{k_i})\:mod\:N$

where $k_i$ is an ephemeral key generated for each m_i

### Encrypt

In [195]:
m = [3011212, 13050919, 8130105, 12] 
k = [36791938, 75575765, 63896758, 876467964] 

r = [pow_mod(g, k_i, N) for k_i in k]
t = [(m[i]*pow_mod(A, k[i], N)) % N for i in range(len(m))]

print("m: ", m)
print("k: ", k)
print("r: ", r)
print("t: ", t)

m:  [3011212, 13050919, 8130105, 12]
k:  [36791938, 75575765, 63896758, 876467964]
r:  [3718412188, 3891282432, 2199126882, 1624742611]
t:  [3869047895, 2075897307, 1983986401, 562059872]


### Decrypt

we multiply each $t_i\:=mA^{k_i}=mg^{k_i*a}$ by inverse of $A^{k_i}$ 

where we get $g^{k_i*a}$ by doing $r_i=g^{k_i}$ to the $a$ power giving us $g^{k_i*a}$ and then finding the inverse and multiply that times $t_i$ giving us the original $m_i$

In [196]:
x = [pow_mod(r_i, a, N) for r_i in r]
print("x: ", x)
y = [pow_mod(x_i, N-2, N) for x_i in x]
print("y: ", y)
z = [(y[i]*t[i]) % N for i in range(len(y))]
print("z: ", z)
assert z == m

x:  [2624927969, 3966818660, 3627167110, 1533568375]
y:  [840574011, 2369070632, 2329243934, 3884955129]
z:  [3011212, 13050919, 8130105, 12]


# Diffie-Hellman

g = p.e. mod N (public)

N = a large prime (public)

a = Alice's secret 1 < a < n

A = $g^a\:mod\:p$

b = Bob's secret number 1 < b < n

B = $g^b\:mod\:p$

Alice computes $B^a\:mod\:N$

Bob computes $A^b\:mod\:N$

Bob and Alice now share the key $g^{ab}$

### Pseudo-Primes

In [36]:
# pseudo-code from https://primes.utm.edu/glossary/page.php?sort=JacobiSymbol
def jacobi(a, n):
    t = 1
    while a != 0:
        while a % 2 == 0:
            a /= 2
            r = n % 8
            if r == 3 or r == 5:
                t = -t
        a, n = n, a
        if a % 4 == n % 4 == 3:
            t = -t
        a %= n
    if n == 1:
        return t
    else:
        return 0

### Solovay-Strassen

In [39]:
def solovay_strassen_prime(n):
    for a in range(2, n):
        if gcd(a, n) != 1:
            continue
        x = pow_mod(a, (n-1)//2, n)
        J = jacobi(a, n)
        if x != (J % n):
            print(f'{a = }')
            print(f'Jacobi: {a}/{n} -> {J}')
            print(f'a^(n-1)/2 -> {x}')
            print(f'{n} is composite')
            return True
    print(f'test is inconclusive')
    return False

In [40]:
solovay_strassen_prime(16)

a = 3
Jacobi: 3/16 -> 1
a^(n-1)/2 -> 11
16 is composite


True

### Miller-Rabin

In [41]:
from random import randint
from scripts.factor import prime_factors
from math import log2
from scripts.gcd import gcd
def get_ms(n):
    ''' returns m, s such that n-1 == 2**s * m'''
    # n_1 = n-1
    n_1 = n-1
    s = 0
    while n_1 % 2 == 0:
        s += 1
        n_1 = n_1//2
    m = n // (2**s)
    return m, s

def mb_prime(n):
    m, s = get_ms(n)
    
    a = randint(2, int(n**(0.5)))
    while gcd(a, n) != 1:
        a = randint(2, int(n**0.5))

    print(f'm = {m}, s = {s}, a={a}')
    # b_0 = a**m mod n
    b = pow(a, m, n)
    print(f'initial {a}**{m} mod {n} == {b}')
    if abs(b) == 1:
        print()
        return False
    
    for i in range(1, s):
        b_n = pow(b, 2, n)
        print(f'{b}**{2} mod {n} == {b_n}')
        if b_n == 560:
            print('test is inconclusive')
            return True
        if b_n == 1:
            print('not prime!')
            return False
        b = b_n
    print('test is inconclusive')
    return True

### Euler Prime

In [43]:
from random import randint
def euler_prime(p, log=False):
    ''' can only return whether a number is not prime or true if we think it could be prime'''
    a = randint(2, p-1)
    for i in range(1000):
        #try 1000 random numbers
        pm = pow_mod(a, p-1, p)
        # print(f'trying {a}**{p-1} mod {p} = {pm}')
        if pm != 1:
            print(f'{a}**{p-1} mod {p} = {pm}')
            print('not equal 1, so not prime!')
            return False
        a = randint(2,p-1)
    if log:
        print()
    print('probably prime')
    return True

# DLP -> Baby-step Giant-step

$a^x \equiv y\:mod\:p$

In [50]:
a = 2
y = 77
p = 101
# x = ?

In [121]:
from math import sqrt
import numpy as np
import pandas as pd

def giant_baby(a, y, p):
    n = int(sqrt(p-1)) + 1
    table = np.ones((n, 2))
    
    #a_initial = pow_mod(a,(-n-1)%p, p)
    a_initial = pow(pow_mod(a, n, p),-1, p)
    table[0][1] = y*a_initial % p
    for i in range(2, n+1):
        # B entry
        table[i-1][0] = pow_mod(a, i, p)
        # G entry
        last_g = table[i-2][1]
        table[i-1][1] = last_g*a_initial % p
    table = table.astype('int32')
    table_df = pd.DataFrame(table, columns=["B[i]", "G[i]"], index=list(range(1,n+1)))
    table_df.index.name = "i"
    return table_df

In [122]:
giant_baby(a, y, p)

,B[i],G[i]
i,,
1,1,28
2,4,1
3,8,83
4,16,21
5,32,26
6,64,37
7,27,41
8,54,70
9,7,53


# Exam

## Problem 1

In [123]:
def print_squares(p):
    l = []
    for i in range(0, (p-1)//2 + 1):
        square = pow(i, 2, p)
        pos_item = (i, square)
        l.append(pos_item)
        if i != 0:
            neg_item = (p-i, square)
            l.append(neg_item)
    squares = set([item[1] for item in l])
    print(f'squares mod {p}: {squares}\n')
    
    square_roots = dict(zip(squares, []*len(squares)))
    for num, square in l:
        if not square_roots.get(square):
            square_roots[square] = []
        square_roots[square].append(num)

    print('dict mapping squares --> list of square roots')
    for square, roots in square_roots.items():
        print(f'{square:2}: {roots}')
    return square_roots

print_squares(13)

squares mod 13: {0, 1, 3, 4, 9, 10, 12}

dict mapping squares --> list of square roots
 0: [0]
 1: [1, 12]
 4: [2, 11]
 9: [3, 10]
 3: [4, 9]
12: [5, 8]
10: [6, 7]


{0: [0], 1: [1, 12], 4: [2, 11], 9: [3, 10], 3: [4, 9], 12: [5, 8], 10: [6, 7]}

## Problem 2

In [141]:
n = 58014043
x = 18059241

In [142]:
# Trial 0

y = 3965911
b = 0
z = 18763
if b==0:
    print("b is 0, Bob responds: ",pow(z,2, n) == (y % n))
else:
    print("b is 1, Bob responds: ", pow(z,2, n) == (x*y % n))

b is 0, Bob responds:  True


In [143]:
# Trial 1

y = 48963613
b = 1
z = 32270779
if b==0:
    print("b is 0, Bob responds: ",pow(z,2, n) == (y % n))
else:
    print("b is 1, Bob responds: ", pow(z,2, n) == (x*y % n))

b is 1, Bob responds:  True


In [144]:
# Trial 2

y = 8302702
b = 1
z = 767595
if b==0:
    print("b is 0, Bob responds: ",pow(z,2, n) == (y % n))
else:
    print("b is 1, Bob responds: ", pow(z,2, n) == (x*y % n))

b is 1, Bob responds:  True


In [146]:
# Trial 3

y = 50014686
b = 0
z = 976657
if b==0:
    print("b is 0, Bob responds: ",pow(z,2, n) == (y % n))
else:
    print("b is 1, Bob responds: ", pow(z,2, n) == (x*y % n))

b is 0, Bob responds:  True


## Problem 4

In [178]:
prime_factors(795)
print((-1)**(1530//2))
print(1531 % 3)
print((-1)**(1530/2*2))
print(1531%5)
print((-1)**(1530//2*52//2))
print(1531 % 53)
print(47%3)
print(jacobi(795, 1531))

-1
1
1.0
1
1
47
2
-1


In [193]:
print(prime_factors(41))
print(prime_factors(79))
print((-1)**((78//2)*(40//2)))
print(79 % 41)
print(prime_factors(38))
print(prime_factors(19))
print((-1)**(((41**2)-1)//8))
print((-1)**((18//2)*(40//2)))
print(41 % 19)
print(19 % 3)
print(jacobi(41,79))

[(41, 1)]
[(79, 1)]
1
38
[(2, 1), (19, 1)]
[(19, 1)]
1
1
3
1
-1
